# imports

In [ ]:
# essential imports
import pandas as pd 
import numpy as np 

In [36]:
# model imports
from xgboost import XGBClassifier as xgb
from sklearn.model_selection import GridSearchCV as gs 

# model tools 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
 

# data import

In [48]:
# Read data
df1 = pd.read_csv('match_model.csv',index_col= 0)

In [52]:
# drop unwanted cols
df = df1.drop(columns= ['Season','Round'])

In [54]:
df

,main,opp,winner,win_type,time
0,Switchback,Banshee,Switchback,JD,0
1,Banshee,Valkyrie,Banshee,JD,0
2,Emulsifier,Banshee,Emulsifier,KO,110
4,Kraken,Beta,Beta,JD,0
5,Beta,Shatter,Beta,JD,0
...,...,...,...,...,...
4950,Lock-Jaw,Yeti,Yeti,JD,0
4951,Chomp,Yeti,Yeti,KO,144
4952,Yeti,Lucky,Yeti,KO,164
4953,Tombstone,Yeti,Tombstone,JD,0


# Get_Dummies

In [51]:
df.head(1)

,main,opp,winner,win_type,time
0,Switchback,Banshee,Switchback,JD,0


In [46]:
df = pd.get_dummies(data = df, columns = ['main', 'opp', 'winner', 'win_type'], drop_first= True )

In [47]:
df.head(1)

,time,main_Atom #94,main_Axe Backwards,main_Axolotl,main_Bad Kitty,main_Bale Spear,main_Bale Spear& Double Dutch,main_Banshee,main_Basilisk,main_BattleSaw,main_Beta,main_Big Dill,main_Bite Force,main_Black Dragon,main_Blacksmith,main_Blade V3,main_Blip,main_Bloodsport,main_Bombshell,main_Breaker Box,main_Bronco,main_Brutus,main_Captain Shrederator,main_Chomp,main_Chronos,main_Claw Viper,main_Cobalt,main_Complete Control,main_Copperhead,main_DUCK!,main_Deadlift,main_Death Roll,main_Deep Six,main_Defender,main_Deviled Egg,main_Doomba,main_Double Dutch,main_Double Jeopardy,main_Dragon King,main_Dragonslayer,...,winner_SMEEEEEEEEEEE,winner_SOW,winner_SawBlaze,winner_Sharkopion,winner_Shatter,winner_Shreddit Bro,winner_Sidewinder,winner_Skorpios,winner_SlamMow,winner_Slap Box,winner_Stinger,winner_SubZero,winner_Switchback,winner_Tantrum,winner_Tantrum 2m59s,winner_TerrorTops,winner_Texas Twister,winner_The Four Horseman,winner_The Ringmaster,winner_Tombstone,winner_Tracer,winner_Triton,winner_Ultimo Destructo,winner_Upper Cut,winner_Valkyrie,winner_WAR? EZ!,winner_Wan Hoo,winner_War Hawk,winner_Warhead,winner_Warrior Dragon,winner_Whiplash,winner_Witch Doctor,winner_Yeti,win_type_Copperhead,win_type_JD,win_type_KO,win_type_Lock-Jaw,win_type_Pain,win_type_TKO,win_type_Tantrum
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


# splitting data

In [41]:
df

,time,main_Atom #94,main_Axe Backwards,main_Axolotl,main_Bad Kitty,main_Bale Spear,main_Bale Spear& Double Dutch,main_Banshee,main_Basilisk,main_BattleSaw,main_Beta,main_Big Dill,main_Bite Force,main_Black Dragon,main_Blacksmith,main_Blade V3,main_Blip,main_Bloodsport,main_Bombshell,main_Breaker Box,main_Bronco,main_Brutus,main_Captain Shrederator,main_Chomp,main_Chronos,main_Claw Viper,main_Cobalt,main_Complete Control,main_Copperhead,main_DUCK!,main_Deadlift,main_Death Roll,main_Deep Six,main_Defender,main_Deviled Egg,main_Doomba,main_Double Dutch,main_Double Jeopardy,main_Dragon King,main_Dragonslayer,...,winner_SMEEEEEEEEEEE,winner_SOW,winner_SawBlaze,winner_Sharkopion,winner_Shatter,winner_Shreddit Bro,winner_Sidewinder,winner_Skorpios,winner_SlamMow,winner_Slap Box,winner_Stinger,winner_SubZero,winner_Switchback,winner_Tantrum,winner_Tantrum 2m59s,winner_TerrorTops,winner_Texas Twister,winner_The Four Horseman,winner_The Ringmaster,winner_Tombstone,winner_Tracer,winner_Triton,winner_Ultimo Destructo,winner_Upper Cut,winner_Valkyrie,winner_WAR? EZ!,winner_Wan Hoo,winner_War Hawk,winner_Warhead,winner_Warrior Dragon,winner_Whiplash,winner_Witch Doctor,winner_Yeti,win_type_Copperhead,win_type_JD,win_type_KO,win_type_Lock-Jaw,win_type_Pain,win_type_TKO,win_type_Tantrum
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4950,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4951,144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
4952,164,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
4953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
train_test_split(test_size = .2, stratify = [] , random_state= 42 )

# scaling

In [ ]:
ss = StandardScaler()

i_data = dataset.data
response = dataset.target
 
# standardization 
scale = object.fit_transform(i_data) 
print(scale)